In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("ChunksMerge").getOrCreate()
print(spark.sparkContext.appName)
#http://localhost:4040/jobs/

22/03/12 22:05:42 WARN Utils: Your hostname, Anishs-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.0.3 instead (on interface en0)
22/03/12 22:05:42 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/03/12 22:05:43 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


ChunksMerge


In [2]:
# npi bhako case ma network and npi bata check garne....
# npi na bhayeko case ma tyo data lai siteuid bata check garne

In [3]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

# Load CHUNK: npi, sort, del-site-uid

In [4]:
filetest1="/Users/anishchapagain/PyAnalysis/sparkcode/bcbsNew/semi.json"

# filetest1="/Users/anishchapagain/PyAnalysis/sparkcode/bcbsNew/bcbs1/bcbs1.json"
# filetest1="/Users/anishchapagain/PyAnalysis/sparkcode/bcbsNew/bcbs3/bcbs3.json"
# filetest1="/Users/anishchapagain/PyAnalysis/sparkcode/bcbsNew/bcbs2/bcbs2.json"
# filetest1="/Users/anishchapagain/PyAnalysis/sparkcode/bcbsNew/bcbs5/bcbs5.json"
# filetest1="/Users/anishchapagain/PyAnalysis/sparkcode/bcbsNew/bcbs4/bcbs4.json"


# filetest1="/Users/anishchapagain/PyAnalysis/sparkcode/bcbs/bcbs_50.json" bcbs_output_1.json
# filetest1="/Users/anishchapagain/PyAnalysis/sparkcode/bcbs/bcbs_40.json"
# filetest1="/Users/anishchapagain/PyAnalysis/sparkcode/bcbs/bcbs_30.json"
# filetest1="/Users/anishchapagain/PyAnalysis/sparkcode/bcbs/bcbs_20.json"
# filetest1="/Users/anishchapagain/PyAnalysis/sparkcode/bcbs/bcbs_10.json"

In [5]:
test1 = spark.read.json(path=filetest1)
type(test1)

pyspark.sql.dataframe.DataFrame

In [6]:
test1.printSchema()

root
 |-- addresses: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- address_string: string (nullable = true)
 |    |    |-- city: string (nullable = true)
 |    |    |-- languages: array (nullable = true)
 |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |-- name: string (nullable = true)
 |    |    |    |    |-- type: string (nullable = true)
 |    |    |-- office_name: string (nullable = true)
 |    |    |-- phones: array (nullable = true)
 |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |-- type: string (nullable = true)
 |    |    |    |    |-- value: string (nullable = true)
 |    |    |-- state: string (nullable = true)
 |    |    |-- street_line_1: string (nullable = true)
 |    |    |-- street_line_2: string (nullable = true)
 |    |    |-- zip: string (nullable = true)
 |-- group_affiliations: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- 

In [7]:
total2 = test1.count()
unique2 = test1.distinct().count()

In [8]:
print(total2)
print(unique2)

7305324
4088577


In [9]:
# test1.select('provider.npi').distinct().count()

In [10]:
# testnpis2 = test1.groupBy("provider.npi").count().orderBy(col('count').desc())
# testnpis2.show(10,truncate=False)

### NEW Schema and Fields: dropping Site_UID

In [11]:
abcdee = test1.select(['provider.*',
#     'provider.facility_name','provider.first_name','provider.gender','provider.last_name','provider.suffix',
#                     'provider.license_number','provider.middle_name','provider.npi','provider.provider_type','provider.title','provider.unparsed_name',
                    array_sort('addresses').alias('addresses'),
                    array_sort('networks').alias('networks'),
                    array_sort('hospital_affiliations').alias('hospital_affiliations'),
                    array_sort('group_affiliations').alias('group_affiliations'),
                    array_sort('specialties').alias('specialties')
                   ])
type(abcdee)

pyspark.sql.dataframe.DataFrame

In [12]:
abcdee.printSchema()

root
 |-- accepting_new_patients: string (nullable = true)
 |-- facility_name: string (nullable = true)
 |-- facility_type: string (nullable = true)
 |-- first_name: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- license_number: string (nullable = true)
 |-- middle_name: string (nullable = true)
 |-- npi: string (nullable = true)
 |-- pcp: boolean (nullable = true)
 |-- pcp_id: string (nullable = true)
 |-- provider_type: string (nullable = true)
 |-- rating: struct (nullable = true)
 |    |-- scale: string (nullable = true)
 |    |-- score: string (nullable = true)
 |-- site_uid: string (nullable = true)
 |-- suffix: string (nullable = true)
 |-- title: string (nullable = true)
 |-- unparsed_name: string (nullable = true)
 |-- addresses: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- address_string: string (nullable = true)
 |    |    |-- city: string (nullable = true)
 |    |    |-- 

In [18]:
abcdee = abcdee.drop('site_uid')

In [19]:
demoabcdee = abcdee.distinct()

In [20]:
totalabcdee = abcdee.count()
uniqueabcdee = demoabcdee.count()

In [21]:
print(totalabcdee)
print(uniqueabcdee)

7305324
4088577


In [22]:
demoabcdee = demoabcdee.withColumn("pcp_id",lit('null'))
demoabcdee = demoabcdee.withColumn("score",lit(''))
demoabcdee = demoabcdee.withColumn("scale",lit(''))
demoabcdee = demoabcdee.withColumn("site_uid",lit(''))
demoabcdee = demoabcdee.withColumn("accepting_new_patients",lit('null'))
demoabcdee = demoabcdee.withColumn("facility_type",lit(''))

In [23]:
demoabcdee.printSchema()

root
 |-- accepting_new_patients: string (nullable = false)
 |-- facility_name: string (nullable = true)
 |-- facility_type: string (nullable = false)
 |-- first_name: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- license_number: string (nullable = true)
 |-- middle_name: string (nullable = true)
 |-- npi: string (nullable = true)
 |-- pcp: boolean (nullable = true)
 |-- pcp_id: string (nullable = false)
 |-- provider_type: string (nullable = true)
 |-- rating: struct (nullable = true)
 |    |-- scale: string (nullable = true)
 |    |-- score: string (nullable = true)
 |-- suffix: string (nullable = true)
 |-- title: string (nullable = true)
 |-- unparsed_name: string (nullable = true)
 |-- addresses: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- address_string: string (nullable = true)
 |    |    |-- city: string (nullable = true)
 |    |    |-- languages: array (nullable = true)
 |

In [24]:
demoabcdee2 = demoabcdee.select('addresses','group_affiliations','hospital_affiliations','networks',
                                struct('facility_name', 'first_name','gender','last_name','site_uid',
                                       'facility_type','accepting_new_patients','pcp_id',
                                   'license_number','middle_name','npi','pcp','provider_type','suffix','title','unparsed_name',
                                   struct('scale','score').alias('rating')
                                  ).alias('provider'),'specialties')

In [25]:
demoabcdee2.printSchema()

root
 |-- addresses: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- address_string: string (nullable = true)
 |    |    |-- city: string (nullable = true)
 |    |    |-- languages: array (nullable = true)
 |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |-- name: string (nullable = true)
 |    |    |    |    |-- type: string (nullable = true)
 |    |    |-- office_name: string (nullable = true)
 |    |    |-- phones: array (nullable = true)
 |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |-- type: string (nullable = true)
 |    |    |    |    |-- value: string (nullable = true)
 |    |    |-- state: string (nullable = true)
 |    |    |-- street_line_1: string (nullable = true)
 |    |    |-- street_line_2: string (nullable = true)
 |    |    |-- zip: string (nullable = true)
 |-- group_affiliations: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- 

In [26]:
demoabcdee2.select('provider.*').show(5,truncate=False)

+-------------+----------+------+--------------+--------+-------------+----------------------+------+--------------+-----------+----------+----+-------------+------+-----+----------------------------+------+
|facility_name|first_name|gender|last_name     |site_uid|facility_type|accepting_new_patients|pcp_id|license_number|middle_name|npi       |pcp |provider_type|suffix|title|unparsed_name               |rating|
+-------------+----------+------+--------------+--------+-------------+----------------------+------+--------------+-----------+----------+----+-------------+------+-----+----------------------------+------+
|null         |Allan     |M     |Sarrat        |        |             |null                  |null  |              |J          |1538296801|null|individual   |LAC   |null |Allan J Sarrat              |{, }  |
|null         |Debra     |F     |Wood          |        |             |null                  |null  |              |           |1447473400|null|individual   |LAC   |nul

In [27]:
# testnpis3 = demoabcdee2.groupBy("provider.npi").count().orderBy(col('count').desc())
# testnpis3.show(10,truncate=False)

In [28]:
demoabcdee2.select('provider').printSchema()

root
 |-- provider: struct (nullable = false)
 |    |-- facility_name: string (nullable = true)
 |    |-- first_name: string (nullable = true)
 |    |-- gender: string (nullable = true)
 |    |-- last_name: string (nullable = true)
 |    |-- site_uid: string (nullable = false)
 |    |-- facility_type: string (nullable = false)
 |    |-- accepting_new_patients: string (nullable = false)
 |    |-- pcp_id: string (nullable = false)
 |    |-- license_number: string (nullable = true)
 |    |-- middle_name: string (nullable = true)
 |    |-- npi: string (nullable = true)
 |    |-- pcp: boolean (nullable = true)
 |    |-- provider_type: string (nullable = true)
 |    |-- suffix: string (nullable = true)
 |    |-- title: string (nullable = true)
 |    |-- unparsed_name: string (nullable = true)
 |    |-- rating: struct (nullable = false)
 |    |    |-- scale: string (nullable = false)
 |    |    |-- score: string (nullable = false)



#### <hr />Original NPI's before SiteUID

## <hr /><hr />

In [29]:
#demoabcdee2.filter(demoabcdee2.provider.npi==1689612954).select('networks','specialties').show(truncate=False)

In [30]:
# cat bcbsOutput4a/part-*.json > bcbsOutput4a/bcbs4.json 

In [31]:
#{"provider":{"first_name":"Jaime","gender":"M","last_name":"Landman","license_number":"","middle_name":"","npi":"1437268505","provider_type":"individual","suffix":"MD","unparsed_name":"Jaime Landman","rating":{}},"networks":[{"name":"BlueCard PPO Basic","tier":""},{"name":"BlueCard PPO/EPO","tier":""},{"name":"BlueCard Traditional","tier":""}],"specialties":[{"name":"Urology"}],"group_affiliations":[{"name":"Prohealth Partners A Medical Group Inc"},{"name":"UCI Department of Urology"},{"name":"UCI Medical Group"},{"name":"Wu Dept of General Surgery"}],"hospital_affiliations":[{"name":"Long Beach Memorial Medical Center"},{"name":"UC Irvine Medical Center"}],"addresses":[{"address_string":"20350 SW Birch St, Newport Beach, CA 92660","city":"Newport Beach","languages":[{"name":"English","type":"primary"},{"name":"Spanish","type":"secondary"}],"office_name":"","phones":[{"type":"phone","value":"7144567005"}],"state":"CA","street_line_1":"20350 SW Birch St","street_line_2":"","zip":"92660"},{"address_string":"2801 Atlantic Ave, Long Beach, CA 90806","city":"Long Beach","languages":[{"name":"English","type":"primary"},{"name":"Spanish","type":"secondary"}],"office_name":"","phones":[{"type":"phone","value":"5629331877"}],"state":"CA","street_line_1":"2801 Atlantic Ave","street_line_2":"","zip":"90806"},{"address_string":"2810 Long Beach Blvd, Long Beach, CA 90806","city":"Long Beach","languages":[{"name":"English","type":"primary"},{"name":"Spanish","type":"secondary"}],"office_name":"","phones":[{"type":"phone","value":"5629331877"}],"state":"CA","street_line_1":"2810 Long Beach Blvd","street_line_2":"","zip":"90806"},{"address_string":"4921 Parkview Pl, St Louis, MO 63110","city":"St Louis","languages":[{"name":"English","type":"primary"},{"name":"Spanish","type":"secondary"}],"office_name":"","phones":[{"type":"phone","value":"3143628200"}],"state":"MO","street_line_1":"4921 Parkview Pl","street_line_2":"","zip":"63110"}]}
#{"provider":{"first_name":"Jaime","gender":"M","last_name":"Landman","license_number":"","middle_name":"","npi":"1437268505","provider_type":"individual","suffix":"MD","unparsed_name":"Jaime Landman","rating":{}},"networks":[{"name":"BlueCard PPO Basic","tier":""},{"name":"BlueCard PPO/EPO","tier":""},{"name":"BlueCard Traditional","tier":""}],"specialties":[{"name":"Urology"}],"group_affiliations":[{"name":"Prohealth Partners A Medical Group Inc"},{"name":"UCI Department of Urology"},{"name":"UCI Medical Group"},{"name":"Wu Dept of General Surgery"}],"hospital_affiliations":[{"name":"Long Beach Memorial Medical Center"},{"name":"UC Irvine Medical Center"}],"addresses":[{"address_string":"20350 SW Birch St, Newport Beach, CA 92660","city":"Newport Beach","languages":[{"name":"English","type":"primary"},{"name":"Spanish","type":"secondary"}],"office_name":"","phones":[{"type":"phone","value":"7145092230"}],"state":"CA","street_line_1":"20350 SW Birch St","street_line_2":"","zip":"92660"},{"address_string":"2801 Atlantic Ave, Long Beach, CA 90806","city":"Long Beach","languages":[{"name":"English","type":"primary"},{"name":"Spanish","type":"secondary"}],"office_name":"","phones":[{"type":"phone","value":"5629331877"}],"state":"CA","street_line_1":"2801 Atlantic Ave","street_line_2":"","zip":"90806"},{"address_string":"2810 Long Beach Blvd, Long Beach, CA 90806","city":"Long Beach","languages":[{"name":"English","type":"primary"},{"name":"Spanish","type":"secondary"}],"office_name":"","phones":[{"type":"phone","value":"5629331877"}],"state":"CA","street_line_1":"2810 Long Beach Blvd","street_line_2":"","zip":"90806"},{"address_string":"4921 Parkview Pl, St Louis, MO 63110","city":"St Louis","languages":[{"name":"English","type":"primary"},{"name":"Spanish","type":"secondary"}],"office_name":"","phones":[{"type":"phone","value":"3143628200"}],"state":"MO","street_line_1":"4921 Parkview Pl","street_line_2":"","zip":"63110"}]}


In [32]:
# demoabcdee2.write.json('file4.json')

In [34]:
demoabcdee2.write.mode("overwrite").option("ignoreNullFields", "false").json("file1.json")

In [36]:
cat outputbcbs/part-*.json > outputbcbs/final.json 

In [37]:
del filetest1
del test1
del demoabcdee
del demoabcdee2

In [38]:
spark.stop()